## Imports

In [31]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from shapely import wkt
from shapely.geometry import Point
import googlemaps

## Data Read in

In [19]:
df = pd.read_csv('CHI_agents_data-Table 1.csv',low_memory=False)

In [17]:
df['geometry'].isna().value_counts()

geometry
False    107944
True        271
Name: count, dtype: int64

In [22]:
# Function to convert WKT format to a Point geometry
def to_point(coord):
    try:
        if isinstance(coord, str) and 'POINT' in coord:
            # Use WKT to convert the string into a Point object
            return wkt.loads(coord)
        else:
            return None  # Return None for invalid or missing coordinates
    except Exception as e:
        return None  # Catch any issues

# Apply the conversion to the 'geometry' column
df['geometry'] = df['geometry'].apply(to_point)

# Now convert the DataFrame to a GeoDataFrame
df_geo = gpd.GeoDataFrame(df, geometry='geometry')

In [25]:
df_geo['geometry'].isna().value_counts()

geometry
False    107944
True        271
Name: count, dtype: int64

In [27]:
df_geo['Full_Address'] = df_geo['Address'] + ' ' + df_geo['Location'] + ' Chicago, IL'

In [32]:
# Retrieve the API key
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [33]:
# Function to geocode an address using googlemaps.Client
def geocode_address_gmaps(address):
    try:
        geocode_result = gmaps_key.geocode(address)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return Point(location['lng'], location['lat'])
        else:
            return None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None

# Apply geocoding function to rows with null geometry
df_geo['geometry'] = df_geo.apply(
    lambda row: geocode_address_gmaps(row['Full_Address']) if row['geometry'] is None else row['geometry'], 
    axis=1
)

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [112]:
# Load the CSV into a regular Pandas DataFrame
alt_df = pd.read_csv('CHI_brokerage_data_fully_geocoded.csv')

# Inspect the column names for duplicates
print(alt_df.columns)

# If multiple "geometry" columns exist, rename or drop them
# Keep the column that actually contains geometry data
if 'geometry' in alt_df.columns:
    alt_df = alt_df.drop_duplicates(subset=['geometry'], keep='first')  # Optional cleanup if needed


if 'geometry' in alt_df.columns:
    alt_gdf = gpd.GeoDataFrame(alt_df)
# # If the geometry is in a different format (e.g., WKT), convert it to GeoDataFrame
# if 'geometry' in alt_df.columns:
#     gdf = gpd.GeoDataFrame(alt_df, geometry=gpd.GeoSeries.from_wkt(alt_df['geometry']), crs="EPSG:4326")
# else:
#     raise ValueError("No valid 'geometry' column found in the file.")

# # `gdf` is now your GeoDataFrame
# print(gdf.head())

Index(['Unnamed: 0', 'mls_id', 'Address', 'Location', 'Zip_Code', 'Price',
       'brokerage_side', 'Final_Brokerage', 'TRD_Note', 'url',
       'brokerage_name', 'brokerage_no.', 'agent', 'SqFt', 'price_per_sq_ft',
       'year_built', 'agent_email', 'geometry', 'data_source_name',
       'within_county', 'Unique', 'Best Match', 'Match?', 'Score',
       'full_address'],
      dtype='object')


/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_81778/2806458054.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  alt_df = pd.read_csv('CHI_brokerage_data_fully_geocoded.csv')


In [119]:
df_geo

,mls_id,Address,Location,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,Brokerage,url,TRD_Note,geometry,SqFt,price_per_sq_ft,year_built,broker_email,data_source_name,within_county,Full_Address,Price_int
0,11805197,691 Sheridan Rd,Winnetka,60093.0,"$12,500,000",buy_agent_1,buyer,Edward Gobbo,Edward Gobbo,Berkshire Hathaway HomeServices Chicago,https://www.redfin.com/IL/Winnetka/691-Sherida...,NaN,POINT (-87.72767 42.11019),"12,500",$892,2005.0,NaN,MRED as Distributed by MLS Grid,True,"691 Sheridan Rd Winnetka Chicago, IL",12500000
1,11805197,691 Sheridan Rd,Winnetka,60093.0,"$12,500,000",list_agent_1,listing,Jena Radnay,Jena Radnay,@properties Christie's International Real Estate,https://www.redfin.com/IL/Winnetka/691-Sherida...,NaN,POINT (-87.72767 42.11019),"12,500",$892,2005.0,NaN,MRED as Distributed by MLS Grid,True,"691 Sheridan Rd Winnetka Chicago, IL",12500000
2,11761344,445 Sheridan Rd,Winnetka,60093.0,"$12,250,000",buy_agent_1,buyer,Jena Radnay,Jena Radnay,@properties Christie's International Real Estate,https://www.redfin.com/IL/Winnetka/445-Sherida...,NaN,POINT (-87.72114 42.10362),"6,174","$1,866",1918.0,NaN,MRED as Distributed by MLS Grid,True,"445 Sheridan Rd Winnetka Chicago, IL",12250000
3,11761344,445 Sheridan Rd,Winnetka,60093.0,"$12,250,000",list_agent_1,listing,Jena Radnay,Jena Radnay,@properties Christie's International Real Estate,https://www.redfin.com/IL/Winnetka/445-Sherida...,NaN,POINT (-87.72114 42.10362),"6,174","$1,866",1918.0,NaN,MRED as Distributed by MLS Grid,True,"445 Sheridan Rd Winnetka Chicago, IL",12250000
4,11902703,65 E Goethe St Unit 8N,CHI - Near North Side,60670.0,"$9,300,000",buy_agent_1,buyer,Mark Icuss,Icuss Group,Compass,https://www.redfin.com/IL/Chicago/65-E-Goethe-...,NaN,POINT (-87.62623 41.90570),"9,000",$869,2000.0,NaN,MRED as Distributed by MLS Grid,True,65 E Goethe St Unit 8N CHI - Near North Side C...,9300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108210,11876519,6950 S Vernon Ave,CHI - Greater Grand Crossing,60637.0,"$100,000",list_agent_1,listing,William Bates Jr,William Bates Jr,EXIT Strategy Realty,https://www.redfin.com/IL/Chicago/6950-S-Verno...,NaN,POINT (-87.61439 41.76788),NaN,NaN,1922.0,NaN,MRED as Distributed by MLS Grid,True,6950 S Vernon Ave CHI - Greater Grand Crossing...,100000
108211,11875755,9549 S Bensley Ave,Jeffery Manor,60617.0,"$100,000",list_agent_1,listing,Willie Jones,Willie Jones,"Associate Brokers, Inc.",https://www.redfin.com/IL/Chicago/9549-S-Bensl...,NaN,POINT (-87.56323 41.72110),937,$143,1950.0,NaN,MRED as Distributed by MLS Grid,True,"9549 S Bensley Ave Jeffery Manor Chicago, IL",100000
108212,12048029,1640 S Millard Ave,CHI - North Lawndale,60623.0,"$100,000",buy_agent_1,buyer,Wilonda Cannon,Wilonda Cannon,Coldwell Banker Realty,https://www.redfin.com/IL/Chicago/1640-S-Milla...,NaN,POINT (-87.71669 41.85778),NaN,NaN,1907.0,NaN,MRED as Distributed by MLS Grid,True,1640 S Millard Ave CHI - North Lawndale Chicag...,100000
108213,11936457,2144 S Drake Ave,CHI - North Lawndale,60623.0,"$100,000",buy_agent_1,buyer,Xitlali Astudillo,Xitlali Astudillo,Luna Realty Group,https://www.redfin.com/IL/Chicago/2144-S-Drake...,NaN,POINT (-87.71390 41.85221),880,$114,1890.0,NaN,MRED as Distributed by MLS Grid,True,"2144 S Drake Ave CHI - North Lawndale Chicago, IL",100000


In [107]:
# Read in neighborhood boundaries
neighborhood_boundaries = gpd.read_file("Boundaries - Community Areas (current).geojson")

# Clean and convert the 'Price' column to integers
df_geo['Price_int'] = df_geo['Price'].str.replace('[\$,]', '', regex=True).astype(int)

# Ensure both GeoDataFrames (df_geo and neighborhood_boundaries) have the same CRS
if df_geo.crs != neighborhood_boundaries.crs:
    df_geo = df_geo.to_crs(neighborhood_boundaries.crs)  # Transform CRS to match

# Perform a spatial join to find which points (brokerages) fall within which community
joined_gdf = gpd.sjoin(df_geo, neighborhood_boundaries, how="inner", predicate='within')

# Group by community and agent/team to calculate counts and total sales
grouped = joined_gdf.groupby(['community', 'Final_Agent/Team']).agg(
    count=('Final_Agent/Team', 'count'),
    total_price=('Price_int', 'sum')
).reset_index()

# Find the agent/team with the highest count for each community
max_count_idx = grouped.groupby("community")['count'].idxmax()
agent_result = grouped.loc[max_count_idx, ['community', 'Final_Agent/Team', 'count', 'total_price']]
agent_result = agent_result.rename(columns={'count': 'agent_count', 'total_price': 'agent_price'})

# Calculate community-level totals
community_result = grouped.groupby('community').agg(
    community_count=('count', 'sum'),
    community_price=('total_price', 'sum')
).reset_index()

# Merge agent results with community-level totals
merged_df = pd.merge(agent_result, community_result, on='community', how='inner')

# Drop unnecessary columns from neighborhood boundaries
columns_to_drop = ['area', 'shape_area', 'perimeter', 'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len']
neighborhood_boundaries = neighborhood_boundaries.drop(columns=columns_to_drop, errors='ignore')

# Merge with neighborhood boundaries
final_merge = neighborhood_boundaries.merge(merged_df, on='community')
final_merge = final_merge.rename(columns={'Final_Agent/Team': 'final_agent_team'})

# Save the result to a GeoJSON file
final_merge.to_file("final_merge.geojson", driver='GeoJSON')


## OLD STUFF

In [79]:
# df_geo['Price_int'] = df_geo['Price'].str.replace('$','').str.replace(',','').astype(int)

In [80]:
# # Read in neighborhood boundaries
# neighborhood_boundaries = gpd.read_file("Boundaries - Community Areas (current).geojson")

# # Ensure both GeoDataFrames (df_geo and neighborhood_boundaries) have the same CRS
# if df_geo.crs != neighborhood_boundaries.crs:
#     df_geo = df_geo.to_crs(neighborhood_boundaries.crs)  # Transform CRS to match

# # Perform a spatial join to find which points (brokerages) fall within which community (multipolygon)
# joined_gdf = gpd.sjoin(df_geo, neighborhood_boundaries, how="inner", predicate='within')

# # Group by community and agent/agent team to get the count and sum of sale prices
# grouped = joined_gdf.groupby(['community', 'Final_Agent/Team']).agg(
#     count=('Final_Agent/Team', 'count'),  # Count of how many times an agent or agent team appears
#     total_price=('Price_int', 'sum')         # Sum sale price
# ).reset_index()

# # Now, `grouped` contains the count and total price grouped by community and agent/team


In [83]:
# print(f'${grouped.total_price.sum():,}')

$26,278,358,580


In [84]:
# # Group by 'community' and find the row with the maximum 'count' in each group
# max_count_idx = grouped.groupby("community")['count'].idxmax()

# # Use the indices to retrieve the rows with the highest 'count' in each community
# agent_result = grouped.loc[max_count_idx, ['community','Final_Agent/Team','count','total_price']]

# agent_result = agent_result.rename(columns={'count':'agent_count','total_price':'agent_price'})

In [85]:
# community_result = grouped.groupby('community').agg({'count':'sum','total_price':'sum'})

# community_result = community_result.rename(columns={'count':'community_count','total_price':'community_price'})

In [86]:
# merged_df = pd.merge(left=agent_result, left_on='community', how='inner',
#                      right=community_result, right_on='community')

In [87]:
# neighborhood_boundaries = neighborhood_boundaries.drop(columns=['area', 'shape_area', 'perimeter', 'area_num_1',
#        'area_numbe', 'comarea_id', 'comarea', 'shape_len'])

In [88]:
# final_merge = neighborhood_boundaries.merge(merged_df, on='community')

In [99]:
# final_merge = final_merge.rename(columns={'Final_Agent/Team':'final_agent_team'})

In [103]:
# final_merge.to_file("final_merge.geojson",driver='GeoJSON')

In [102]:
# final_merge.columns

Index(['community', 'geometry', 'final_agent_team', 'agent_count',
       'agent_price', 'community_count', 'community_price'],
      dtype='object')

In [90]:
# final_merge['community_price'].max()

3345481313

In [91]:
# final_merge['community_price'].min()

780000

In [92]:
# final_merge.head(1)

,community,geometry,Final_Agent/Team,agent_count,agent_price,community_count,community_price
0,DOUGLAS,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",Xiaohuang Wu,6,1457500,231,63789252


In [95]:
# final_merge.columns

Index(['community', 'geometry', 'Final_Agent/Team', 'agent_count',
       'agent_price', 'community_count', 'community_price'],
      dtype='object')

In [77]:
# for i, row in final_merge.iterrows():
#     if row['community_price'] > row['agent_price']:
#         print(f"{row['community']} -- ${row['community_price'] - row['agent_price']:,}")
#         print(f"{row['community']} -- {row['community_price'] / row['community_count']:}%")
#         print(f"{row['Final_Agent/Team']} -- ${row['agent_price'] / row['brokerage_count']:,}")
#         print('------------')
#     else:
#         print('Not great!')

DOUGLAS -- $62,331,752
DOUGLAS -- 276143.94805194804%
Xiaohuang Wu -- $242,916.66666666666
------------
OAKLAND -- $25,834,000
OAKLAND -- 378097.1830985916%
Lane Chesebro -- $336,966.6666666667
------------
FULLER PARK -- $11,842,822
FULLER PARK -- 294591.0%
Roger Valdez -- $265,000.0
------------
GRAND BOULEVARD -- $168,567,482
GRAND BOULEVARD -- 442955.056122449%
Naja Morris -- $563,433.3333333334
------------
KENWOOD -- $137,259,633
KENWOOD -- 469941.3194888179%
Robert Sullivan -- $1,092,444.4444444445
------------
LINCOLN SQUARE -- $557,339,463
LINCOLN SQUARE -- 626392.6255555556%
Biazar Group -- $583,081.8181818182
------------
WASHINGTON PARK -- $25,571,700
WASHINGTON PARK -- 375021.5189873418%
Biazar Group -- $1,013,750.0
------------
HYDE PARK -- $173,981,394
HYDE PARK -- 337016.3384030418%
Jinhong Wang -- $137,050.0
------------
WOODLAWN -- $158,172,766
WOODLAWN -- 430397.56135770236%
Biazar Group -- $741,055.5555555555
------------
ROGERS PARK -- $286,136,868
ROGERS PARK -- 2

In [93]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCountyBrokersNeighborhoods_12_4_2024


In [94]:
# df_geo.to_csv("backup_geocoded_csv.csv")